<h1>IBM Data Science Final Project</h1>

<h2>Introduction: Business Problem</h2>

<b>One of my friends , Robinson, is relocating to India and is interested in setting up a Goan Restaurant in Chennai.He has been a sea food lover since his childhood. He is looking at locations where presence of Resturants is minimal and in particular there are no Goan Cuisine Restaurants. He has asked me to leverage my data science skills and come up with ideal locations for his new venture i.e. Goan Restaurant.</b>

<h3>Data</h3>

<b> Based on my friend's requirements, below are the factors that needs to be considered</b>
1. Number of existing restaurants in each neighburhood
2. Number of Goan restaurants in each neighborhood
3. Proximity and distance

<b>To achieve the above:</b>
1. First we need to get the list of neighborhoods in Chennai city
https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai
2. We need to scrape the wikipedia page containing the neighborhoods details ans fetch them
3. Then for each neighborhood, get latitude and longitude using Google Maps API
4. Then, using the four square API,get the restaurants and Goan Restaurant in each neighborhood
5. We will then use k-means clustering method to group the neighborhoods based on how similar they are.
6. The tools for EDA will be pandas, folium, google API, matplotlib.


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<b>Scrape data from wikipedia page to get the neighborhoods</b>

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text
data

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Category:Suburbs of Chennai - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Suburbs_of_Chennai","wgTitle":"Suburbs of Chennai","wgCurRevisionId":881969379,"wgRevisionId":881969379,"wgArticleId":34457266,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Neighbourhoods in Chennai"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wg

In [3]:
from bs4 import BeautifulSoup

In [4]:
soup=BeautifulSoup(data,'lxml')

In [5]:
neighborhood=[]
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhood.append(row.text)

In [6]:
df_neighborhood=pd.DataFrame({'Neighborhood':neighborhood})
df_neighborhood.head()

,Neighborhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Ayanavaram


In [7]:
def get_lat_long(neighborhood):
    geolocator = Nominatim(user_agent="chn_explorer")
    latitude=None
    longitude=None
    location = geolocator.geocode(neighborhood,timeout=25)
    if location is not None:
        latitude = location.latitude
        longitude = location.longitude
    return latitude, longitude


In [8]:
address_geocode_list=[]
for address in neighborhood:
    latitude, longitude = get_lat_long(address)
    address_geocode_list.append([address,latitude,longitude])
#address_geocode_list
df_geo_list = pd.DataFrame(address_geocode_list,columns=['Neighborhood','Latitude','Longitude'])
df_geo_list = df_geo_list.dropna()
df_geo_list.head()
    

,Neighborhood,Latitude,Longitude
0,Alandur,12.994373,80.194284
1,Anna Nagar,11.170349,77.351114
2,"Ashok Nagar, Chennai",13.040073,80.215925
3,Assisi Nagar,13.164610,80.233000
4,Ayanavaram,13.094616,80.235410
